# LSTM Time Series - Stock Price Prediction
## Part 2 - Data Preprocessing
In this notebook, we load the filtered data set file containing the historical prices of Google stocks for the last five years and preprocess it.

> **INPUT**: The filtered data set file of the last five years stock prices for Google, as filtered in the previous phase. <br/>
> **OUTPUT**: The preprocessed and transformed data, split into training, validation, and testing subsets, and stored in an intermediary location for the training phase.